# MD5

In [1]:
import time

In [2]:
#message = "The quick brown foxx jumps over the lazy dog very quickly and very brownly!  After all it was a quick fox and brown too!"
message = "A"

```
"A"
7FC56270E7A70FA81A5935B72EACBE29

"AA"
3B98E2DFFC6CB06A89DCB0D5C60A0206

"B"
9D5ED678FE57BCCA610140957AFAB571

"BB"
9D3D9048DB16A7EEE539E93E3618CBE7

"MD5"
7F138A09169B250E9DCB378140907378
```

### Step 1 - Append Padding Bits

In [3]:
message_len = len(message)
message_len_bits = message_len * 8
print("Message Length : "+str(message_len)+" bytes | "+str(message_len_bits)+" bits")

Message Length : 1 bytes | 8 bits


In [4]:
message_b = message.encode('utf-8')

In [5]:
padding_len=56-message_len%64
padding_len=64 if (padding_len==0) else padding_len
print("Padding Length : "+str(padding_len)+"bytes | "+str(padding_len * 8)+" bits")

Padding Length : 55bytes | 440 bits


In [6]:
message_mod448 = message_b + b'\x80' + b'\x00' * (padding_len-1)
print("Padded Message :\n"+str(message_mod448))
print("\nlength(paddedMessage)      : "+str(len(message_mod448))+" bytes | "+str(len(message_mod448*8))+" bits\nlength(paddedMessage) % 64 : "+str(len(message_mod448)%64)+" bytes | "+str((len(message_mod448)%64) * 8)+" bits" )

Padded Message :
b'A\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

length(paddedMessage)      : 56 bytes | 448 bits
length(paddedMessage) % 64 : 56 bytes | 448 bits


### Step 2 - Append Length

In [7]:
# Append Length
processed_message=message_mod448+(message_len_bits%2**64).to_bytes(8,byteorder='little')
print("LSB64(len(unPaddedMessage)) : "+str((message_len_bits%2**64).to_bytes(8,byteorder='big')))
print("length( paddedMessage | LSB64(len(unPaddedMessage)) ) : "+str(len(processed_message))+" bytes | "+str(len(processed_message)*8)+" bits")
print("\nPadded Message | LSB64(len(unPaddedMessage)) :\n"+str(processed_message))

LSB64(len(unPaddedMessage)) : b'\x00\x00\x00\x00\x00\x00\x00\x08'
length( paddedMessage | LSB64(len(unPaddedMessage)) ) : 64 bytes | 512 bits

Padded Message | LSB64(len(unPaddedMessage)) :
b'A\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x08\x00\x00\x00\x00\x00\x00\x00'


### Step 3 - Initilize MD Buffer

In [8]:
A = 0x67452301
B = 0xEFCDAB89
C = 0x98BADCFE
D = 0x10325476

### Step 4 - Process Message in 16-Word Blocks

In [9]:
def F(X, Y, Z):
    return ((X&Y) | ((~X) & Z))

def G(X, Y, Z):
    return ((X&Z) | (Y & (~Z)))

def H(X, Y, Z):
    return (X^Y^Z)

def I(X, Y, Z):
    return ( Y^(X|(~Z)) )

In [10]:
# Sine Table
sine_T=[0xD76AA478, 0xE8C7B756, 0x242070DB, 0xC1BDCEEE, 0xF57C0FAF, 0x4787C62A, 0xA8304613, 0xFD469501, 0x698098D8, 0x8B44F7AF, 0xFFFF5BB1, 0x895CD7BE, 0x6B901122, 0xFD987193, 0xA679438E, 0x49B40821, 0xF61E2562, 0xC040B340, 0x265E5A51, 0xE9B6C7AA, 0xD62F105D, 0x02441453, 0xD8A1E681, 0xE7D3FBC8, 0x21E1CDE6, 0xC33707D6, 0xF4D50D87, 0x455A14ED, 0xA9E3E905, 0xFCEFA3F8, 0x676F02D9, 0x8D2A4C8A, 0xFFFA3942, 0x8771F681, 0x6d9d6122, 0xFDE5380C, 0xA4BEEA44, 0x4BDECFA9, 0xF6BB4B60, 0xBEBFBC70, 0x289B7EC6, 0xEAA127FA, 0xD4EF3085, 0x04881D05, 0xD9D4D039, 0xE6DB99E5, 0x1FA27CF8, 0xC4AC5665, 0xF4292244, 0x432AFF97, 0xAB9423A7, 0xFC93A039, 0x655B59C3, 0x8F0CCC92, 0xFFEFF47D, 0x85845DD1, 0x6FA87E4F, 0xFE2CE6E0, 0xA3014314, 0x4E0811A1, 0xF7537E82, 0xBD3AF235, 0x2AD7D2BB, 0xEB86D391]

In [11]:
print(hex(sine_T[35]),hex(sine_T[36]),hex(sine_T[37]))

0xfde5380c 0xa4beea44 0x4bdecfa9


In [12]:
# Rotate Left
def rotl(x,s):
    return ( (x<<s) | x>>(32-s))

In [13]:
# Shift Table
R1_s=[7,12,17,22]*4
R2_s=[5, 9,14,20]*4
R3_s=[4,11,16,23]*4
R4_s=[6,10,15,21]*4

# K table
R1_k = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
R2_k = [1, 6, 11, 0, 5, 10, 15, 4, 9, 14, 3, 8, 13, 2, 7, 12]
R3_k = [5, 8, 11, 14, 1, 4, 7, 10, 13, 0, 3, 6, 9, 12, 15, 2]
R4_k = [0, 7, 14, 5, 12, 3, 10, 1, 8, 15, 6, 13, 4, 11, 2, 9]

In [14]:
def bytereverse(num32):
    rev_byte=0;
    for i in range(0,16):
        #print(hex(num32)+" "+hex(rev_byte))
        rev_byte = rev_byte << 8
        
        low_order_byte = num32 & 0xFF
        rev_byte = rev_byte | low_order_byte
        
        num32 = num32 >> 8
    return rev_byte

In [15]:
## Round Functions
def round1(a, b, c, d, X, k, sine_i, s):
    Xk = int.from_bytes(X[4*k:4*k+4],byteorder='little')
    #print(hex(F(b,c,d)))
    FN = F(b,c,d)
    a = (a + FN + Xk + sine_i) & 0xFFFFFFFF
    a = ((rotl(a , s)& 0xFFFFFFFF) + b) & 0xFFFFFFFF
    print("     R1 | K = "+"{:2d}".format(k)+" | s = "+"{:2d}".format(s)+" | sine = "+"{:8x}".format(sine_i)+" | {:8x}".format(FN))
    return a 

def round2(a, b, c, d, X, k, sine_i, s):
    Xk = int.from_bytes(X[4*k:4*k+4],byteorder='little')
    #print(hex(G(b,c,d)))
    FN = G(b,c,d)
    a = (a + FN + Xk + sine_i) & 0xFFFFFFFF
    a = ((rotl(a , s)& 0xFFFFFFFF) + b) & 0xFFFFFFFF
    print("     R2 | K = "+"{:2d}".format(k)+" | s = "+"{:2d}".format(s)+" | sine = "+"{:8x}".format(sine_i)+" | {:8x}".format(FN))
    return a 

def round3(a, b, c, d, X, k, sine_i, s):
    Xk = int.from_bytes(X[4*k:4*k+4],byteorder='little')
    #print(hex(H(b,c,d)))
    FN = H(b,c,d)
    a = (a + FN + Xk + sine_i) & 0xFFFFFFFF
    a = ((rotl(a , s)& 0xFFFFFFFF) + b) & 0xFFFFFFFF
    print("     R3 | K = "+"{:2d}".format(k)+" | s = "+"{:2d}".format(s)+" | sine = "+"{:8x}".format(sine_i)+" | {:8x}".format(FN))
    return a

def round4(a, b, c, d, X, k, sine_i, s):
    Xk = int.from_bytes(X[4*k:4*k+4],byteorder='little')
    #print(hex(I(b,c,d)))
    FN = I(b,c,d)
    a = (a + FN + Xk + sine_i) & 0xFFFFFFFF
    a = ((rotl(a , s)& 0xFFFFFFFF) + b) & 0xFFFFFFFF
    print("     R4 | K = "+"{:2d}".format(k)+" | s = "+"{:2d}".format(s)+" | sine = "+"{:8x}".format(sine_i)+" | {:8x}".format(FN))
    return a

In [16]:
i=0
k=0
X=processed_message[i:i+64]
print("X :\n"+str(X))
print("X_: "+str(X[4*k:4*k+4]))
Xk = int.from_bytes(X[4*k:4*k+4],byteorder='big')
print("X[k] : "+str(Xk))
print(hex(Xk))

X :
b'A\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x08\x00\x00\x00\x00\x00\x00\x00'
X_: b'A\x80\x00\x00'
X[k] : 1098907648
0x41800000


In [17]:
for i in range(0,len(processed_message),64):
    print("PROCESSING bytes "+str(i)+"..."+str(i+64))
    X  = processed_message[i:i+64]
    print("     "+str(X)+" \n")
    AA = A
    BB = B
    CC = C
    DD = D
    
    #round 1
    print("*** ROUND 1 ***")
    A = round1(A, B, C, D, X, R1_k[0], sine_T[0], R1_s[0])
    #print(hex(A))
    D = round1(D, A, B, C, X, R1_k[1], sine_T[1], R1_s[1])
    #print(hex(D))
    C = round1(C, D, A, B, X, R1_k[2], sine_T[2], R1_s[2])
    #print(hex(C))
    B = round1(B, C, D, A, X, R1_k[3], sine_T[3], R1_s[3])
    #print(hex(D))
    
    A = round1(A, B, C, D, X, R1_k[4], sine_T[4], R1_s[4])
    #print(hex(A))
    D = round1(D, A, B, C, X, R1_k[5], sine_T[5], R1_s[5])
    #print(hex(D))
    C = round1(C, D, A, B, X, R1_k[6], sine_T[6], R1_s[6])
    #print(hex(C))
    B = round1(B, C, D, A, X, R1_k[7], sine_T[7], R1_s[7])
    #print(hex(B))
    
    A = round1(A, B, C, D, X, R1_k[8], sine_T[8], R1_s[8])
    #print(hex(A))
    D = round1(D, A, B, C, X, R1_k[9], sine_T[9], R1_s[9])
    #print(hex(D))
    C = round1(C, D, A, B, X, R1_k[10], sine_T[10], R1_s[10])
    #print(hex(C))
    B = round1(B, C, D, A, X, R1_k[11], sine_T[11], R1_s[11])
    #print(hex(B))
    
    A = round1(A, B, C, D, X, R1_k[12], sine_T[12], R1_s[12])
    #print(hex(A))
    D = round1(D, A, B, C, X, R1_k[13], sine_T[13], R1_s[13])
    #print(hex(D))
    C = round1(C, D, A, B, X, R1_k[14], sine_T[14], R1_s[14])
    #print(hex(C))
    B = round1(B, C, D, A, X, R1_k[15], sine_T[15], R1_s[15])
    #print(hex(B))
    
    print("\n*** ROUND 2 ***")
    A = round2(A, B, C, D, X, R2_k[0], sine_T[16], R2_s[0])
    #print(hex(A))
    D = round2(D, A, B, C, X, R2_k[1], sine_T[17], R2_s[1])
    #print(hex(D))
    C = round2(C, D, A, B, X, R2_k[2], sine_T[18], R2_s[2])
    #print(hex(C))
    B = round2(B, C, D, A, X, R2_k[3], sine_T[19], R2_s[3])
    #print(hex(D))
    
    A = round2(A, B, C, D, X, R2_k[4], sine_T[20], R2_s[4])
    #print(hex(A))
    D = round2(D, A, B, C, X, R2_k[5], sine_T[21], R2_s[5])
    #print(hex(D))
    C = round2(C, D, A, B, X, R2_k[6], sine_T[22], R2_s[6])
    #print(hex(C))
    B = round2(B, C, D, A, X, R2_k[7], sine_T[23], R2_s[7])
    #print(hex(B))
    
    A = round2(A, B, C, D, X, R2_k[8], sine_T[24], R2_s[8])
    #print(hex(A))
    D = round2(D, A, B, C, X, R2_k[9], sine_T[25], R2_s[9])
    #print(hex(D))
    C = round2(C, D, A, B, X, R2_k[10], sine_T[26], R2_s[10])
    #print(hex(C))
    B = round2(B, C, D, A, X, R2_k[11], sine_T[27], R2_s[11])
    #print(hex(B))
    
    A = round2(A, B, C, D, X, R2_k[12], sine_T[28], R2_s[12])
    #print(hex(A))
    D = round2(D, A, B, C, X, R2_k[13], sine_T[29], R2_s[13])
    #print(hex(D))
    C = round2(C, D, A, B, X, R2_k[14], sine_T[30], R2_s[14])
    #print(hex(C))
    B = round2(B, C, D, A, X, R2_k[15], sine_T[31], R2_s[15])
    #print(hex(B))
    
    print("\n*** ROUND 3 ***")
    A = round3(A, B, C, D, X, R3_k[0], sine_T[32], R3_s[0])
    #print(hex(A))
    D = round3(D, A, B, C, X, R3_k[1], sine_T[33], R3_s[1])
    #print(hex(D))
    C = round3(C, D, A, B, X, R3_k[2], sine_T[34], R3_s[2])
    #print(hex(C))
    B = round3(B, C, D, A, X, R3_k[3], sine_T[35], R3_s[3])
    #print(hex(D))
    
    A = round3(A, B, C, D, X, R3_k[4], sine_T[36], R3_s[4])
    #print(hex(A))
    D = round3(D, A, B, C, X, R3_k[5], sine_T[37], R3_s[5])
    #print(hex(D))
    C = round3(C, D, A, B, X, R3_k[6], sine_T[38], R3_s[6])
    #print(hex(C))
    B = round3(B, C, D, A, X, R3_k[7], sine_T[39], R3_s[7])
    #print(hex(B))
    
    A = round3(A, B, C, D, X, R3_k[8], sine_T[40], R3_s[8])
    #print(hex(A))
    D = round3(D, A, B, C, X, R3_k[9], sine_T[41], R3_s[9])
    #print(hex(D))
    C = round3(C, D, A, B, X, R3_k[10], sine_T[42], R3_s[10])
    #print(hex(C))
    B = round3(B, C, D, A, X, R3_k[11], sine_T[43], R3_s[11])
    #print(hex(B))
    
    A = round3(A, B, C, D, X, R3_k[12], sine_T[44], R3_s[12])
    #print(hex(A))
    D = round3(D, A, B, C, X, R3_k[13], sine_T[45], R3_s[13])
    #print(hex(D))
    C = round3(C, D, A, B, X, R3_k[14], sine_T[46], R3_s[14])
    #print(hex(C))
    B = round3(B, C, D, A, X, R3_k[15], sine_T[47], R3_s[15])
    #print(hex(B))
    
    print("\n*** ROUND 4 ***")
    A = round4(A, B, C, D, X, R4_k[0], sine_T[48], R4_s[0])
    #print(hex(A))
    D = round4(D, A, B, C, X, R4_k[1], sine_T[49], R4_s[1])
    #print(hex(D))
    C = round4(C, D, A, B, X, R4_k[2], sine_T[50], R4_s[2])
    #print(hex(C))
    B = round4(B, C, D, A, X, R4_k[3], sine_T[51], R4_s[3])
    #print(hex(D))
    
    A = round4(A, B, C, D, X, R4_k[4], sine_T[52], R4_s[4])
    #print(hex(A))
    D = round4(D, A, B, C, X, R4_k[5], sine_T[53], R4_s[5])
    #print(hex(D))
    C = round4(C, D, A, B, X, R4_k[6], sine_T[54], R4_s[6])
    #print(hex(C))
    B = round4(B, C, D, A, X, R4_k[7], sine_T[55], R4_s[7])
    #print(hex(B))
    
    A = round4(A, B, C, D, X, R4_k[8], sine_T[56], R4_s[8])
    #print(hex(A))
    D = round4(D, A, B, C, X, R4_k[9], sine_T[57], R4_s[9])
    #print(hex(D))
    C = round4(C, D, A, B, X, R4_k[10], sine_T[58], R4_s[10])
    #print(hex(C))
    B = round4(B, C, D, A, X, R4_k[11], sine_T[59], R4_s[11])
    #print(hex(B))
    
    A = round4(A, B, C, D, X, R4_k[12], sine_T[60], R4_s[12])
    #print(hex(A))
    D = round4(D, A, B, C, X, R4_k[13], sine_T[61], R4_s[13])
    #print(hex(D))
    C = round4(C, D, A, B, X, R4_k[14], sine_T[62], R4_s[14])
    #print(hex(C))
    B = round4(B, C, D, A, X, R4_k[15], sine_T[63], R4_s[15])
    #print(hex(B))
    
    A = (A + AA) & 0xFFFFFFFF
    B = (B + BB) & 0xFFFFFFFF
    C = (C + CC) & 0xFFFFFFFF
    D = (D + DD) & 0xFFFFFFFF
    #time.sleep(1)

PROCESSING bytes 0...64
     b'A\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x08\x00\x00\x00\x00\x00\x00\x00' 

*** ROUND 1 ***
     R1 | K =  0 | s =  7 | sine = d76aa478 | 98badcfe
     R1 | K =  1 | s = 12 | sine = e8c7b756 | bddadb8a
     R1 | K =  2 | s = 17 | sine = 242070db | ad488be8
     R1 | K =  3 | s = 22 | sine = c1bdceee | a3704374
     R1 | K =  4 | s =  7 | sine = f57c0faf | f3d877e0
     R1 | K =  5 | s = 12 | sine = 4787c62a | a32d07b6
     R1 | K =  6 | s = 17 | sine = a8304613 | 2225c0d7
     R1 | K =  7 | s = 22 | sine = fd469501 | 60d9405e
     R1 | K =  8 | s =  7 | sine = 698098d8 | b98f6aa6
     R1 | K =  9 | s = 12 | sine = 8b44f7af | b94e48e0
     R1 | K = 10 | s = 17 | sine = ffff5bb1 | fb6e5d60
     R1 | K = 11 | s = 22 | sine = 895cd7be | 78754997
     R1 | K = 12 | s =  7 | sine = 6

In [18]:
output_int = D<<96 | C <<64 | B << 32 | A
#print([hex(D),hex(C),hex(B),hex(A)])
output = "{:x} {:x} {:x} {:x}".format(D,C,B,A)
print("OUTPUT1:   "+output)
print("OUTPUT2: "+hex(output_int))
print("OUTPUT2: "+hex(bytereverse(output_int)))

OUTPUT1:   29beac2e b735591a a80fa7e7 7062c57f
OUTPUT2: 0x29beac2eb735591aa80fa7e77062c57f
OUTPUT2: 0x7fc56270e7a70fa81a5935b72eacbe29


In [19]:
"{:x} {:x} {:x} {:x}".format(A,B,C,D)

'7062c57f a80fa7e7 b735591a 29beac2e'